# WARNING
**Please make sure to "COPY AND EDIT NOTEBOOK" to use compatible library dependencies! DO NOT CREATE A NEW NOTEBOOK AND COPY+PASTE THE CODE - this will use latest Kaggle dependencies at the time you do that, and the code will need to be modified to make it work. Also make sure internet connectivity is enabled on your notebook**

# Preliminaries

Write requirements to file, anytime you run it, in case you have to go back and recover Kaggle dependencies. **MOST OF THESE REQUIREMENTS WOULD NOT BE NECESSARY FOR LOCAL INSTALLATION**

Requirements are hosted for each notebook in the companion github repo, and can be pulled down and installed here if needed. Companion github repo is located at https://github.com/azunre/transfer-learning-for-nlp


In [1]:
!pip freeze > kaggle_image_requirements.txt

In [2]:
# get critical dependencies for our purposes
!pip install tensor2tensor
!git clone https://github.com/jessevig/bertviz.git

     |████████████████████████████████| 1.4 MB 4.6 MB/s 
     |████████████████████████████████| 352 kB 18.8 MB/s 
     |████████████████████████████████| 46 kB 3.0 MB/s 
     |████████████████████████████████| 366 kB 21.4 MB/s 
     |████████████████████████████████| 119 kB 24.7 MB/s 
     |████████████████████████████████| 365 kB 22.2 MB/s 
     |████████████████████████████████| 981 kB 29.6 MB/s 
     |████████████████████████████████| 191 kB 38.5 MB/s 
     |████████████████████████████████| 649 kB 35.2 MB/s 
     |████████████████████████████████| 370 kB 40.4 MB/s 
     |████████████████████████████████| 590 kB 49.6 MB/s 
     |████████████████████████████████| 11.8 MB 47.9 MB/s 
     |████████████████████████████████| 179 kB 40.6 MB/s 
     |████████████████████████████████| 44.7 MB 21 kB/s 
  Created wheel for bz2file: filename=bz2file-0.98-py3-none-any.whl size=6882 sha256=8a5611e546f99978a9ab666831a690a23219b0a3c5bd66d9b408d37dc5f19423
  Stored in directory: /root/.cache/pip/w

# BERT Self-Attention Visualization

In [3]:
from transformers import BertTokenizer, BertModel # transformers BERT tokenizer and model

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [4]:
model = BertModel.from_pretrained('bert-base-uncased', output_attentions=True) # load uncased bert model, making sure to output attention
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True) # load uncased bert tokenizer

**Load some required javascript libraries for displaying visualization in notebook**

In [5]:
%%javascript
require.config({
  paths: {
      d3: '//cdnjs.cloudflare.com/ajax/libs/d3/3.4.8/d3.min',
      jquery: '//ajax.googleapis.com/ajax/libs/jquery/2.0.0/jquery.min',
  }
});

<IPython.core.display.Javascript object>

In [6]:
from bertviz.bertviz import head_view # bertviz attention head visualization method

# Function for displaying the multiheaded attention
def show_head_view(model, tokenizer, sentence):
    input_ids = tokenizer.encode(sentence, return_tensors='pt', add_special_tokens=True) # be sure to use PyTorch tensors with bertviz 
    attention = model(input_ids)[-1] # get attention layer
    tokens = tokenizer.convert_ids_to_tokens(list(input_ids[0]))    
    head_view(attention, tokens) # call the internal bertviz method to display self-attention

In [7]:
# you can always take a very detailed look at the model like this
print(model)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

How to tokenize the input and convert to ready ingestible form by model?

In [8]:
sentence = "He didnt want to talk about cells on the cell phone because he considered it boring"
inputs = tokenizer.encode(sentence, return_tensors='tf', add_special_tokens=True) # changing return_tensors to "pt" would return PyTorch tensors
print(inputs)

tf.Tensor(
[[  101  2002  2134  2102  2215  2000  2831  2055  4442  2006  1996  3526
   3042  2138  2002  2641  2009 11771   102]], shape=(1, 19), dtype=int32)


In [9]:
tokens = tokenizer.convert_ids_to_tokens(list(inputs[0])) # Extract sample of batch index 0 from inputs list of lists
print(tokens)

['[CLS]', 'he', 'didn', '##t', 'want', 'to', 'talk', 'about', 'cells', 'on', 'the', 'cell', 'phone', 'because', 'he', 'considered', 'it', 'boring', '[SEP]']


Now, actually show the self-attention

In [10]:
show_head_view(model, tokenizer, sentence)

<IPython.core.display.Javascript object>

# English-Twi Translation Example

We load the Helsinki-NLP model, trained on the JW300 corpus, for the Twi language - a popular language in Ghana. 

This is an encoder-decoder architecture very similar to that in the original "Attention is all you need" transformers paper

Over a thousand languages are available - https://huggingface.co/Helsinki-NLP

The high-resource translation models are pretty good. The low resource models - for a language like Twi - are a good baseline.

In [11]:
from transformers import MarianMTModel, MarianTokenizer

model = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-en-tw")
tokenizer = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-tw")

/opt/conda/lib/python3.7/site-packages/transformers/tokenization_utils.py:831: FutureWarning: Parameter max_len is deprecated and will be removed in a future release. Use model_max_length instead.
  category=FutureWarning,
/opt/conda/lib/python3.7/site-packages/transformers/tokenization_marian.py:97: UserWarning: Recommended: pip install mosestokenizer
  warnings.warn("Recommended: pip install mosestokenizer")


In [12]:
text = "My name is Paul" # Input English Sentence to be translated
inputs = tokenizer.encode(text, return_tensors="pt") # Encode to input token ids
outputs = model.generate(inputs) # Generate output token ids
decoded_output = [tokenizer.convert_ids_to_tokens(int(outputs[0][i])) for i in range(len(outputs[0]))] # Decode output token ids to actual output tokens
print("Original Sentence:")
print(text)
print("Translation (good):")
print(decoded_output)

Original Sentence:
My name is Paul
Translation (good):
['<pad>', '▁Me', '▁din', '▁de', '▁Paul']


In [13]:
text = "How are things?"
inputs = tokenizer.encode(text, return_tensors="pt")
outputs = model.generate(inputs)
decoded_output = [tokenizer.convert_ids_to_tokens(int(outputs[0][i])) for i in range(len(outputs[0]))]
print("Original Sentence:")
print(text)
print("Translation (bad):")
print(decoded_output)

Original Sentence:
How are things?
Translation (bad):
['<pad>', '▁Ɔkwan', '▁bɛn', '▁so', '▁na', '▁nneɛma', '▁te', '▁saa', '?']


The translation is roughly "In which way are things like this?" which while semantically similar, conveys a different meaning. 

The semantic similarity however shows that his model is a decent start for further training/transfer. As shown in the following code, rephrasing lightly gets the right translation.

In [14]:
text = "How are you?"
inputs = tokenizer.encode(text, return_tensors="pt")
outputs = model.generate(inputs)
decoded_output = [tokenizer.convert_ids_to_tokens(int(outputs[0][i])) for i in range(len(outputs[0]))]
print("Original Sentence:")
print(text)
print("Translation (good):")
print(decoded_output)

Original Sentence:
How are you?
Translation (good):
['<pad>', '▁Wo', '▁ho', '▁te', '▁dɛn', '?']


In [15]:
# remove bertviz folder for saving notebook
!rm -r bertviz